# From Expectations to Synthetic Data generation

## 2. The synthetic data generation


In [2]:
from IPython.display import JSON, display_json
import json

import pandas as pd

In [3]:
dataset_name = "BankChurn"
data = pd.read_csv('BankChurners.csv')

f = open(f'.profile_{dataset_name}.json')
json_profile = json.load(f)
json_profile = json.loads(json_profile)

Let's leverage `pandas-profiling` automated detection of data types to separate the columns by dtype for the synthesis process.

In [4]:
num_cols = [col for col, val in json_profile['variables'].items() if val['type']=='Numeric' and col!='cardio']
cat_cols = [col for col, val in json_profile['variables'].items() if val['type']=='Categorical' and col!='cardio']

print(f'Number of categorical: {len(num_cols)}, Number of numerical: {len(cat_cols)}')

Number of categorical: 17, Number of numerical: 6


In [5]:
data

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.999910
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.999940
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.999980
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.999870
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.999980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,...,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,0.000191,0.999810
10123,710638233,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,...,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,0.995270,0.004729
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,0.997880,0.002118
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,0.996710,0.003294


### Prepare the data for synthesis

After checking the warnings generated by `pandas-profiling` we are able to understand that the cardio dataset is generally well-behaved, meaning that we can leverage the standard data preparation performed by the synthesizer architectures:
- **Numerical columns** - Standard Scaler, important to ensure a faster convergence of the models and ease of results reproducibility.
- **Categorical columns** - Label Encoder

Because we aim to generate synthetic data as close to the original one, it is recommended to not perform any outlier treatment of feature engineering.

<div class="alert alert-block alert-warning">
<b>Note:</b> The selection and use of data transformations for synthesis will vary based on the dataset and synthetic data generation approach.
</div>

As the *Attrition_Flag* is an imbalanced variables, we are going to consider the use of a conditional architecture, CWGAN-GP, while consitioning the training to this flag.

<p style="text-align:center;"><img src="img/cgan.jpeg" alt = "test pic" width="500" height="200"></p>

[Image source](https://arxiv.org/abs/1411.1784)

In [6]:
#The cardio dataset is pretty balanced in what concerns the target variable.
#But not only - variables like gender are quite balanced even when compared under the context of the target
data['Attrition_Flag'].value_counts()

Existing Customer    8500
Attrited Customer    1627
Name: Attrition_Flag, dtype: int64

### Data Preparation##

In [7]:
#dropping the ClientNum column as it is unique.
p_data = data.drop('CLIENTNUM', axis=1)

#prepating the conditional column
p_data['Attrition_Flag'] = p_data['Attrition_Flag'].astype('category')
p_data['Attrition_Flag'] = p_data['Attrition_Flag'].cat.codes

In [8]:
#Update the columns to be synth
cat_cols = [col for col in cat_cols if col != 'Attrition_Flag']
num_cols = [col for col in num_cols if col != 'CLIENTNUM']

In [9]:
cat_cols

['Gender',
 'Education_Level',
 'Marital_Status',
 'Income_Category',
 'Card_Category']

In [10]:
num_cols

['Customer_Age',
 'Dependent_count',
 'Months_on_book',
 'Total_Relationship_Count',
 'Months_Inactive_12_mon',
 'Contacts_Count_12_mon',
 'Credit_Limit',
 'Total_Revolving_Bal',
 'Avg_Open_To_Buy',
 'Total_Amt_Chng_Q4_Q1',
 'Total_Trans_Amt',
 'Total_Trans_Ct',
 'Total_Ct_Chng_Q4_Q1',
 'Avg_Utilization_Ratio',
 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2']

### Training a synthesizer

In [19]:
from ydata_synthetic.synthesizers.regular import RegularSynthesizer
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters

In [36]:
## Setting the architecture hyperparameters
noise_dim = 32
dim = 128
batch_size = 64

#Defined as per the literature on CWGAN
beta_1 = 0.5
beta_2 = 0.9

log_step = 100
epochs = 500 + 1
learning_rate = 0.0001
models_dir = '../cache'

model_parameters = ModelParameters(batch_size=batch_size,
                                   lr=learning_rate,
                                   betas=(beta_1, beta_2),
                                   noise_dim=noise_dim,
                                   layers_dim=dim)

train_args = TrainParameters(epochs=epochs,
                             cache_prefix='',
                             sample_interval=log_step,
                             label_dim=-1,
                             labels=(0,1))

In [37]:
#Init the synthesizer model
#n_critic sets the number of updates of the critic network per adversarial training
synth = RegularSynthesizer(modelname='cwgangp', model_parameters=model_parameters, n_critic=5)

#Model training
synth.fit(data=p_data,
          label_cols=["Attrition_Flag"],
          train_arguments=train_args,
          num_cols=num_cols, cat_cols=cat_cols)

2022-10-12 01:20:16.738658: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Number of iterations per epoch: 159


  0%|          | 1/501 [00:04<40:00,  4.80s/it]

Epoch: 0 | critic_loss: -0.36268314719200134 | gen_loss: 0.12872254848480225


  0%|          | 2/501 [00:07<29:07,  3.50s/it]

...

Epoch: 498 | critic_loss: -0.4687464237213135 | gen_loss: 0.14555180072784424


100%|█████████▉| 500/501 [23:51<00:02,  2.38s/it]

Epoch: 499 | critic_loss: -0.36978569626808167 | gen_loss: 0.3225882649421692


100%|██████████| 501/501 [23:54<00:00,  2.86s/it]

Epoch: 500 | critic_loss: -0.5806481838226318 | gen_loss: 0.6157317161560059


In [14]:
#Saving the trained synthesizer
from ydata_synthetic.synthesizers.regular import RegularSynthesizer
synth = RegularSynthesizer.load(f'{dataset_name}_synth.pkl')

2022-10-12 01:56:11.197312: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 01:56:11.227037: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-10-12 01:56:11.227072: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-12 01:56:11.267782: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [15]:
cond_array = p_data[["Attrition_Flag"]]

#Generating a sample with the same size and conditional configuration as the original dataset
synth_sample = synth.sample(cond_array)

#Saving the synthetic sample as CSV
synth_sample.to_csv(f'synth_{dataset_name}.csv')

## Summary & Next Steps

Now that we were able to generate successfully our synthetic data sample we need to assess whether the output data of our synthesizer as enough quality. The quality of synthetic data can be translated into *Fidelity* and *Utility*.

In the next notebook we will explore the *Fidelity* of our dataset through:
- Synthetic data profiling vs Real data profiling
- Running real data suit of expectations